In [6]:
import re
import time
import subprocess
import pandas as pd
from selenium import webdriver
from IPython.display import display, HTML
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--start-maximized")
options.add_argument("--disable-dev-shm-usage")

In [8]:
output = subprocess.run(
    ["/Applications/Google Chrome.app/Contents/MacOS/Google Chrome", "--version"],
    capture_output=True,
    text=True
).stdout.strip()

version_number = re.search(r"(\d+\.\d+\.\d+)", output)
chrome_version = version_number.group(1)
print(chrome_version)

141.0.7390


In [10]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version=chrome_version).install()), options=options)
driver.get("https://gourmetmarketthailand.com/farmhouse?srsltid=AfmBOoqOkoerVNxy_3nu2FhkxO2R2WucImnE6nR9-hAbbKy2pyDzHBxK")
time.sleep(3)

while True:
    try:
        load_more = driver.find_element(By.CSS_SELECTOR, "button.button-load-more")
        load_more.click()
        time.sleep(2)  # wait for more products to load
    except:
        break  # no more load buttons

products = driver.find_elements(By.CSS_SELECTOR, "a.item-card-detail") 

for product in products:
    product_name = product.find_element(By.CSS_SELECTOR, "div.mb-16.name").text 
    product_price = product.find_element(By.CSS_SELECTOR, "div.price-per-unit").text
    print(product_name)
    print(product_price)
    print("-------------------------")

driver.quit()

ฟาร์มเฮ้าส์ ขนมปังทาหน้ามอคค่า 50 กรัม
฿9.00/ชิ้น
-------------------------
ฟาร์มเฮ้าส์ ขนมปังฮอตดอกไส้หมูหยอง 50ก.
฿12.00/ชิ้น
-------------------------
ฟาร์มเฮ้าส์ ขนมปังแซนด์วิช 480ก.
฿40.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังรอยัลโฮลวีตเนื้อนุ่มสไลด์หนา 285ก.
฿40.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังปอนด์ 240ก.
฿21.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังสไลด์ขอบ 220ก.
฿30.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังโฮลวีต 250ก.
฿24.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังโฮลวีต 500ก.
฿44.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังรอยัลเบรดชนิดแผ่นเนื้อนุ่มสไลด์หนา 275ก.
฿36.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังโฮลวีตชนิดเนื้อละเอียด 250ก.
฿24.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังรอยัลธัญพืช 12 ชนิด 285ก.
฿44.00/แพ็ค
-------------------------
ฟาร์มเฮ้าส์ ขนมปังกลิ่นนมฮอกไกโด 240ก.
฿30.00/แพ็ค
-------------------------
ขนมปังบัตเตอร์สก็อต 250ก.
฿42.00/แพ็ค
-------------------------
รอ

In [15]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version=chrome_version).install()), options=options)
driver.get("https://gourmetmarketthailand.com/farmhouse?srsltid=AfmBOoqOkoerVNxy_3nu2FhkxO2R2WucImnE6nR9-hAbbKy2pyDzHBxK")
time.sleep(3)

while True:
    try:
        load_more = driver.find_element(By.CSS_SELECTOR, "button.button-load-more")
        driver.execute_script("arguments[0].click();", load_more)  # safer click with JS
        load_more.click()
        time.sleep(5)
    except:
        break

products = driver.find_elements(By.CSS_SELECTOR, "a.item-card-detail")

data = []

for product in products:
    product_name = product.find_element(By.CSS_SELECTOR, "div.mb-16.name").text
    product_price = product.find_element(By.CSS_SELECTOR, "div.price-per-unit").text

    #match = re.match(r"^(ฟาร์มเฮ้าส์)?\s*(.*?)\s*(\d+)\s*(ก\.?|กรัม|g)$", product_name)
    match = re.match(r"^(ฟาร์มเฮ้าส์)?\s*(.*?)\s*(\d+)\s*(ก\.?|กรัม|g)\.*", product_name)

    if match:
        company = match.group(1) if match.group(1) else "ฟาร์มเฮ้าส์"
        name = match.group(2).strip()
        weight = match.group(3)
        weight_unit = "g"
    else:
        company = "ฟาร์มเฮ้าส์"
        name = product_name
        weight, weight_unit = None, None

    price_match = re.match(r"(฿)\s*([\d.,]+)\/(.+)", product_price)
    if price_match:
        currency = price_match.group(1)
        price = price_match.group(2)
        unit = price_match.group(3)
    else:
        currency, price, unit = None, None, None

    data.append({
        "company": company,
        "product_name": name,
        "product_weight": weight,
        "weight_unit": weight_unit,
        "currency": currency,
        "product_price": price,
        "unit": unit
    })

driver.quit()

df = pd.DataFrame(data)
df

,company,product_name,product_weight,weight_unit,currency,product_price,unit
0,ฟาร์มเฮ้าส์,ขนมปังทาหน้ามอคค่า,50,g,฿,9.00,ชิ้น
1,ฟาร์มเฮ้าส์,ขนมปังฮอตดอกไส้หมูหยอง,50,g,฿,12.00,ชิ้น
2,ฟาร์มเฮ้าส์,ขนมปังแซนด์วิช,480,g,฿,40.00,แพ็ค
3,ฟาร์มเฮ้าส์,ขนมปังรอยัลโฮลวีตเนื้อนุ่มสไลด์หนา,285,g,฿,40.00,แพ็ค
4,ฟาร์มเฮ้าส์,ขนมปังปอนด์,240,g,฿,21.00,แพ็ค
5,ฟาร์มเฮ้าส์,ขนมปังสไลด์ขอบ,220,g,฿,30.00,แพ็ค
6,ฟาร์มเฮ้าส์,ขนมปังโฮลวีต,250,g,฿,24.00,แพ็ค
7,ฟาร์มเฮ้าส์,ขนมปังโฮลวีต,500,g,฿,44.00,แพ็ค
8,ฟาร์มเฮ้าส์,ขนมปังรอยัลเบรดชนิดแผ่นเนื้อนุ่มสไลด์หนา,275,g,฿,36.00,แพ็ค
9,ฟาร์มเฮ้าส์,ขนมปังโฮลวีตชนิดเนื้อละเอียด,250,g,฿,24.00,แพ็ค
